In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from ast import literal_eval
import json

In [ ]:
URL = 'https://zapisy.ii.uni.wroc.pl/courses/semester/347'

class Courses:

    def __init__(self, url, load_from_url=False):
        self.url = url
        self.main_url = "https://zapisy.ii.uni.wroc.pl"
        self.load_from_url = load_from_url
        
        if self.load_from_url:
            self.courses, self.filters = self.read_elements()
            self.read_all_courses_org()
            self.save_to_json()
        else:
            self.load_from_json()

        self.courses_kind = ['Wykłady','Ćwiczenia','Ćwiczenio-pracownie',
                'Pracownie','Repetytoria','Projekty','Seminaria',]
        self.people = self.get_all_people()
        self.courses_names = self.get_all_courses()

        self.effect_to_int = {v: int(k) for k,v in self.filters['allEffects'].items()}
        self.tag_to_int = {v: int(k) for k,v in self.filters['allTags'].items()}
        self.type_to_int = {v: int(k) for k,v in self.filters['allTypes'].items()}
        # dict_keys(['allEffects', 'allTags', 'allOwners', 'allTypes'])

    def read_elements(self):
        page = requests.get(self.url)
        soup = BeautifulSoup(page.content, "html.parser")
        results_courses = soup.find(id="courses-data")
        for i in results_courses:
            courses = json.loads(i.string)
            break
        
        results_filters = soup.find(id="filters-data")
        for i in results_filters:
            filters = json.loads(i.string)
            break

        return courses, filters

    def read_one_course(self, input_course):
        page = requests.get(self.main_url + input_course['url'])
        soup = BeautifulSoup(page.content, "html.parser")
        course_org = {}

        # description if needed
        for elem in soup.find_all("div", {"class": "description"}):
            course_description = elem.findAll('span')[0].text
            # print(course_description.split()[:50])
            course_org['description'] = ' '.join(course_description.split()[:100])
            # print(course_description.split()[:100])
            break

        for elem in soup.find_all("div", {"class": "table-responsive tutorial"}):
            kind = elem.findAll('h3')[0].text.strip()
            
            lecturer = [p.text.strip() for p in elem.find_all("a", {"class": "person"})]
            
            hours = elem.findAll('span')
            hours =[h.text.strip() for i, h in enumerate(hours) if i%2 == 1]
            
            hours_lect = list(zip(lecturer, hours))
            
            course_org[kind] = hours_lect
        return course_org

    def read_all_courses_org(self):
        for i, course in enumerate(self.courses):
            org = self.read_one_course(course)
            self.courses[i] = self.courses[i] | org
            # print(self.courses[i])
            # return

    def save_to_json(self):
        with open('courses_all.json', 'w') as fh:
            json.dump(self.courses, fh, indent=4)
        with open('filters_all.json', 'w') as fh:
            json.dump(self.filters, fh, indent=4)

    def load_from_json(self):
        with open('courses_all.json', 'r') as fh:
            self.courses = json.load(fh)
        with open('filters_all.json', 'r') as fh:
            self.filters = json.load(fh)

    def get_courses_type(self, input_type):
        if type(input_type) != int:
            input_type = self.type_to_int[input_type]
        res = []
        for course in self.courses:
            if course['courseType'] == input_type:
                res.append(course['name'])
        return res

    def get_courses_tag(self, tag):
        if type(tag) != int:
            tag = self.tag_to_int[tag]
        res = []
        for course in self.courses:
            if tag in course['tags']: 
                res.append(course['name'])
        return res

    def get_courses_effect(self, effect):
        if type(effect) != int:
            effect = self.effect_to_int[effect]
        res = []
        for course in self.courses:
            if effect in course['effects']: 
                res.append(course['name'])
        return res

    def get_courses_person(self, person):
        res = []
        for course in self.courses:
            for kind in self.courses_kind:
                if kind in course.keys():
                    courses_person = [i[1] for i in course[kind] if i[0] == person]
                    if len(courses_person) > 0:
                        res.append([course['name'], kind, courses_person])
        return res
    
    def get_course_instances(self, course_name):
        res = []
        for course in self.courses:
            if course['name'] == course_name:
                keys = set(course.keys()) & set(self.courses_kind)
                for k in keys:
                    res.append([k, course[k]])
        return res            
    
    def get_all_people(self):
        res = set()
        for course in self.courses:
            keys = set(course.keys()) & set(self.courses_kind)
            for k in keys:
                res = res | set([i[0] for i in course[k]])
        
        return res

    def get_all_courses(self):
        res = []
        for course in self.courses:
            res.append(course['name'])
        return res  


In [ ]:
# get data from pdf manually, scraping is an overkill here
from collections import namedtuple

FirstSplit = namedtuple('FirstSplit', 'licencjanckie inżynierskie')
SecondSplit = namedtuple('SecondSplit', 'trzysemestralnych czterosemestralnych')
DegreeeData = namedtuple('DegreeeData', 'compulsory_courses other_requirements ects_numbers')

MIN_MULTI = 'co najmniej 28 ECTS za przedmioty z tagiem RPiS, IO, PiPO, ASK, SO, SK, BD; min. jeden przedmiot na tag'
LANG = 'lektoraty na poziomie B2+ oraz najmniej jeden przedmiot z grupy I2 lub S w języku angielskim'

class StudyRules:

    def __init__(self):

        self.first_degree = self.data_first_degree()
        self.second_degree = self.data_second_degree()
        self.data_science = self.data_data_science()

    def data_first_degree(self):
        compulsory_courses = ['Analiza matematyczna','Logika dla informatyków','Algebra','Metody programowania',
                        'Analiza numeryczna','Matematyka dyskretna (L)','Algorytmy i struktury danych (L)']

        other_requirements = {
            'przedmioty informatyczne (I)': FirstSplit('co najmniej 54 ECTS', 'co najmniej 66 ECTS, w tym 12 za przedmioty Iinż'),
            'kursy narzędzi informatyki (K)': FirstSplit('brak', 'co najmniej 10 ECTS za kursy Kinż'),
            'projekt programistyczny (P)': FirstSplit('indywidualny lub zespołowy', 'zespołowy'),
            'łącznie przedmioty O+I+K+P': FirstSplit('co najmniej 140 ECTS', 'co najmniej 170 ECTS'),
            'przedmioty I1, IInż, K1 i Kinż tematyczne': FirstSplit(MIN_MULTI, MIN_MULTI),
            'proseminarium (PS)': FirstSplit('co najmniej jedno', 'co najmniej jedno'),
            'praktyka zawodowa': FirstSplit('co najmniej 3 ECTS (3 tygodnie/75h)', 'co najmniej 4 ECTS (4 tygodnie/100h)'),
            'przedmioty humanistyczno-społeczne (HS)': FirstSplit('co najmniej 5 ECTS', 'co najmniej 5 ECTS'),
            'przedmioty oznaczone OWI': FirstSplit('co najmniej 1 ECTS', 'co najmniej 1 ECTS'),
            'przedmioty oznaczone E': FirstSplit('brak', 'co najmniej 2 ECTS'),
            'Język obcy': FirstSplit('angielski na poziomie B2', 'angielski na poziomie B2'),
        }
        ects_numbers = {
            'min. liczba ECTS': FirstSplit('180 ECTS', '210 ECTS'),
            'za pracę dyplomową i egzamin': FirstSplit('10 ECTS', '10 ECTS'),
        }

        return DegreeeData(compulsory_courses, other_requirements, ects_numbers)

    def data_second_degree(self):
        compulsory_courses = ['Języki formalne i złożoność obliczeniowa']

        other_requirements = {
            'przedmioty I2': SecondSplit('co najmniej 36 ECTS', 'co najmniej 54 ECTS'),
            'przedmioty AZ': SecondSplit('co najmniej 12 ECTS', 'co najmniej 12 ECTS'),
            'przedmioty oznaczone SY, PD i NG': SecondSplit('co najmniej 12 ECTS', 'co najmniej 12 ECTS'),
            'przedmioty z grup I2 oraz S oznaczone tym samym tagiem': SecondSplit('co najmniej 15 ECTS', 'co najmniej 15 ECTS'),
            'seminaria (S)': SecondSplit('co najmniej 6 ECTS', 'co najmniej 9 ECTS'),
            'przedmioty humanistyczno-społeczne': SecondSplit('co najmniej 5 ECTS', 'co najmniej 5 ECTS'),
            'język angielski': SecondSplit(LANG, LANG),
        }
        ects_numbers = {
            'min. liczba ECTS': SecondSplit('90 ECTS', '120 ECTS'),
            'za pracę dyplomową i egzamin': SecondSplit('20 ECTS', '20 ECTS'),
        }

        return DegreeeData(compulsory_courses, other_requirements, ects_numbers)
        
    def data_data_science(self):
        pass

# study_rules = StudyRules()

